**1. Indexing Embeddings**

In [5]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

# 1. Load the webpage and extract the relevant content (post title, headers, content)
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
docs = WebBaseLoader(web_paths=["https://lilianweng.github.io/posts/2023-06-23-agent/"], bs_kwargs={"parse_only": bs4_strainer}).load()

# 2. Chunk the text for embedding
recursive_text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=200, separators=["\n\n", "\n", " ", ""])
page_content = [doc.page_content for doc in docs]
chunks = []
for content in page_content:
    chunks.extend(recursive_text_splitter.split_text(content))

# 3. Embed the content and store it in FAISS vector store
embed = OllamaEmbeddings(model="nomic-embed-text")
embeddings = embed.embed_documents(chunks)  # Generate embeddings
vectorstore = FAISS.from_embeddings(list(zip(chunks, embeddings)), embedding=embed)  # Save embeddings to FAISS vector store

#### Conversational RAG.

**2. Create History-Aware Retriever Chain**

In [6]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_ollama import ChatOllama

# Define the contextualize question prompt
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [("system", contextualize_q_system_prompt),
     MessagesPlaceholder("chat_history"),
     ("human", "{input}")]
)

llm = ChatOllama(model="deepseek-r1:1.5b", temperature=0.8)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

history_aware_retriever = create_history_aware_retriever(llm,retriever, contextualize_q_prompt) # Create history-aware retriever

**3. Create Document Chain to Process Retrieved Context**

In [7]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser

# Load LLM model
llm = ChatOllama(model="deepseek-r1:1.5b", temperature=0.8)

# Define the prompt for QA
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [("system", qa_system_prompt), MessagesPlaceholder("chat_history"), ("human", "{input}")]
)

# Create the document processing chain
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)


**4. Combine Retrieval and Document Processing Chains**

In [8]:
from langchain.chains import create_retrieval_chain
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

**5. Stateful Chat History Management**

In [9]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# State management for chat history
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Wrap the retrieval and processing chain with chat history
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

**6. Generation: Final Combined Chain with Chat History**

In [12]:
import uuid

session_id = str(uuid.uuid4())  # Generates a unique identifier
response=conversational_rag_chain.invoke({"input": "What is Agent?"},config={"configurable": {"session_id": session_id}})["answer"]
print(response)

<think>

</think>

An **Agent** is an autonomous entity that perceives its environment and takes actions to achieve specific goals. The term "agent" was introduced by Richard E. Markov and Åke Madsen in their 1963 paper "On the Solution of Partially Observable Markov Decision Processes." An agent operates within a dynamic environment, where it can sense the current state (perception), select an action (action selection), and execute that action (行动执行) to influence the environment. The goal is typically to maximize some cumulative reward or achieve a desired outcome.

### Key Components of an Agent:
1. **Agent State**: Represents the current situation in the environment.
2. **Action Space**: The set of possible actions the agent can take at any given state.
3. **Perception**: The information the agent has about its environment, often through sensors or other means.
4. **Reward Signal**: Provides feedback on the outcome of an action (e.g., positive or negative).
5. **Policy**: A strategy

**Check Chat_history**

In [14]:
print(store)

{'b84b8f6e-0a1c-46cf-8d10-595afc88b0a2': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}), AIMessage(content='<think>\n\n</think>\n\nTask decomposition refers to the process of breaking down a complex task into smaller, more manageable subtasks or components. This approach allows individuals or systems to tackle each part separately, reducing overwhelm and improving efficiency.\n\n### Key Aspects of Task Decomposition:\n\n1. **Simplification**: By dividing tasks, complexity is reduced, making them easier to understand and execute.\n2. **Improved Management**: Subtasks can be prioritized and scheduled more effectively, ensuring clarity on priorities.\n3. **Enhanced Retention**: Breakdown helps in retaining better skills by focusing on specific aspects before integrating them into a larger system.\n4. **Enhanced Creativity**: Breaking tasks into smaller pieces can foster creativity as solutions often emer